## Usage Checker

This notebook demonstrates how to check the usage of node definitions for node libraries.

The utility scans one or more libraries
    * Each definition is examined to see if it is implemented as a node graph. 
        * If it is the list of nodes it uses is returned.
        * For each node that is used a list of node definitions that use it is returned.
    * A list of definitions which are not implemented as graphs as well as a list of nodes that are missing implementations are returned.

The utility also provides usage rate for node definitions and a hint of complexity of a node definition which uses other nodes by providing a count of
the number of unique nodes it uses.

In [16]:
import MaterialX as mx
import mtlxutils.mxfile as mxf
from IPython.display import display_markdown

stdlib = mx.createDocument()
searchPath = mx.getDefaultDataSearchPath()
libraryFolders = mx.getDefaultDataLibraryFolders() 
try:
    libFiles = mx.loadLibraries(libraryFolders, searchPath, stdlib)
    print('Loaded %s standard library definitions for version %s' % (len(stdlib.getNodeDefs()), mx.getVersionString()))
except mx.Exception as err:
    print('Failed to load standard library definitions: "', err, '"')

doc = mx.createDocument()
doc.importLibrary(stdlib)


Loaded 750 standard library definitions for version 1.39.0


## Utility Methods

In [17]:

def getNodeDefUsage(doc, libraryFilter=None):
    
    # List of nodes used by another node
    nodes_use = dict()
    nodes_used_by = dict()
    nongraph_nodes = []
    unimplemented = []

    for nodedef in doc.getNodeDefs():
        found = True
        if libraryFilter:
            found = False
            for lib in libraryFilter:
                if lib in nodedef.getSourceUri():
                    found = True
                    break
        if not found:
            continue

        nodename = nodedef.getNodeString()
        impl = nodedef.getImplementation()
        if impl:
            nodes_use_list = [] 
            for node in impl.getChildrenOfType(mx.Node):
                #print('scan:', node.getName())
                nd = node.getNodeDef()
                if nd:
                    ns = nd.getNodeString()
                    if ns not in nodes_use_list:                    
                        nodes_use_list.append(ns)

                    # Look for ns key in nodes_used_by dictionary
                    if ns not in nodes_used_by:
                        nodes_used_by[ns] = []

                    if nodename not in nodes_used_by[ns]:
                        nodes_used_by[ns].append(nodename)
                else:
                    print('Skip unknown node type: ', node.getName())

            if len(nodes_use_list) > 0:
                nodes_use[nodename] = nodes_use_list
            else:
                if nodename not in nongraph_nodes:
                    nongraph_nodes.append(nodename)
        else:
            if nodename not in unimplemented:
                unimplemented.append(nodename)

    # Sort 
    nongraph_nodes = sorted(nongraph_nodes)
    unimplemented = sorted(unimplemented)
    nodes_use = {k: v for k, v in sorted(nodes_use.items(), key=lambda item: len(item[1]), reverse=True)}
    nodes_used_by = {k: v for k, v in sorted(nodes_used_by.items(), key=lambda item: len(item[1]), reverse=True)}

    return nodes_use, nodes_used_by, nongraph_nodes, unimplemented

def printNodeDefUsage(nodes_use, nodes_used_by, nongraph_nodes, unimplemented):
    #
    result = '| Node Definition | Uses |\n| --- | --- |'
    for nu in nodes_use:
        if (len(nodes_use[nu]) > 0):
            result += '\n| %s | %d nodes: %s |' % (nu, len(nodes_use[nu]), ', '.join(sorted(nodes_use[nu])))
    display_markdown(result, raw=True)

    result = '| Node Definition | Used by |\n| --- | --- |'
    for nu in nodes_used_by:
        if (len(nodes_used_by[nu]) > 0):
            result += '\n| %s | %d nodes: %s |' % (nu, len(nodes_used_by[nu]), ', '.join(sorted(nodes_used_by[nu])) )
    display_markdown(result, raw=True)

    if len(nongraph_nodes) > 0:
        result = '| Non-Graph Nodes: %d |\n| --- |\n' % (len(nongraph_nodes))
        result += '|' + ', '.join(nongraph_nodes) + '|'
        display_markdown(result, raw=True)

    if (len(unimplemented) > 0):
        result = '| Unimplemented Nodes %d |\n| --- |\n' % (len(unimplemented))
        #for nu in unimplemented:
        #    result += '\n| %s |' % (nu)
        result += '|' + ', '.join(unimplemented) + '|'
        display_markdown(result, raw=True)
    

## Node Usages Results for "Standard Library"

In [18]:
nodes_use, nodes_used_by, nongraph_nodes, unimplemented = getNodeDefUsage(doc, ['stdlib', 'nprlib'])

printNodeDefUsage(nodes_use, nodes_used_by, nongraph_nodes, unimplemented)

| Node Definition | Uses |
| --- | --- |
| triplanarprojection | 13 nodes: absval, add, clamp, combine2, divide, dotproduct, extract, image, multiply, normalize, power, separate3, switch |
| unifiednoise2d | 12 nodes: add, cellnoise2d, combine3, fractal3d, multiply, noise2d, range, rotate2d, separate2, subtract, switch, worleynoise2d |
| gooch_shade | 11 nodes: add, divide, dotproduct, max, mix, multiply, normal, normalize, power, reflect, viewdirection |
| tiledcircles | 11 nodes: add, circle, combine2, convert, divide, ifequal, ifgreater, max, modulo, multiply, subtract |
| tiledcloverleafs | 11 nodes: add, cloverleaf, combine2, convert, divide, ifequal, ifgreater, max, modulo, multiply, subtract |
| tiledhexagons | 11 nodes: add, combine2, convert, divide, hexagon, ifequal, ifgreater, max, modulo, multiply, subtract |
| unifiednoise3d | 10 nodes: add, cellnoise3d, fractal3d, multiply, noise3d, range, rotate3d, subtract, switch, worleynoise3d |
| hexagon | 10 nodes: absval, clamp, combine2, constant, dotproduct, ifgreater, min, multiply, sqrt, subtract |
| crosshatch | 10 nodes: add, combine2, convert, ifequal, ifgreater, line, max, modulo, multiply, subtract |
| grid | 9 nodes: absval, add, convert, ifequal, ifgreater, min, modulo, multiply, subtract |
| range | 8 nodes: absval, clamp, divide, ifequal, multiply, power, remap, sign |
| line | 7 nodes: clamp, distance, divide, dotproduct, ifgreater, multiply, subtract |
| checkerboard | 6 nodes: dotproduct, floor, mix, modulo, multiply, subtract |
| refract | 6 nodes: add, dotproduct, ifgreater, multiply, sqrt, subtract |
| facingratio | 5 nodes: absval, dotproduct, ifequal, invert, multiply |
| cloverleaf | 5 nodes: add, circle, combine2, max, subtract |
| place2d | 5 nodes: add, divide, rotate2d, subtract, switch |
| hsvadjust | 5 nodes: add, convert, hsvtorgb, multiply, rgbtohsv |
| colorcorrect | 5 nodes: colorcorrect, combine3, combine4, separate3, separate4 |
| tiledimage | 4 nodes: divide, image, multiply, subtract |
| randomfloat | 4 nodes: cellnoise2d, combine2, convert, range |
| circle | 4 nodes: dotproduct, ifgreater, multiply, subtract |
| safepower | 4 nodes: absval, multiply, power, sign |
| ramp4 | 3 nodes: clamp, extract, mix |
| trianglewave | 3 nodes: absval, modulo, subtract |
| reflect | 3 nodes: dotproduct, multiply, subtract |
| contrast | 3 nodes: add, multiply, subtract |
| overlay | 3 nodes: combine4, overlay, separate4 |
| randomcolor | 2 nodes: convert, randomcolor |
| bump | 2 nodes: heighttonormal, normalmap |
| distance | 2 nodes: magnitude, subtract |
| saturate | 2 nodes: luminance, mix |
| convert | 2 nodes: convert, surface_unlit |
| extract | 2 nodes: switch, swizzle |
| noise2d | 1 nodes: noise2d |
| noise3d | 1 nodes: noise3d |
| fractal3d | 1 nodes: fractal3d |
| smoothstep | 1 nodes: smoothstep |
| separate2 | 1 nodes: swizzle |
| separate3 | 1 nodes: swizzle |
| separate4 | 1 nodes: swizzle |

| Node Definition | Used by |
| --- | --- |
| multiply | 24 nodes: checkerboard, circle, colorcorrect, contrast, crosshatch, facingratio, gooch_shade, grid, hexagon, hsvadjust, line, overlay, randomfloat, range, reflect, refract, safepower, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, triplanarprojection, unifiednoise2d, unifiednoise3d |
| subtract | 21 nodes: checkerboard, circle, cloverleaf, colorcorrect, contrast, crosshatch, distance, grid, hexagon, line, overlay, place2d, reflect, refract, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, trianglewave, unifiednoise2d, unifiednoise3d |
| add | 16 nodes: cloverleaf, colorcorrect, contrast, crosshatch, gooch_shade, grid, hsvadjust, place2d, randomcolor, refract, tiledcircles, tiledcloverleafs, tiledhexagons, triplanarprojection, unifiednoise2d, unifiednoise3d |
| dotproduct | 9 nodes: checkerboard, circle, facingratio, gooch_shade, hexagon, line, reflect, refract, triplanarprojection |
| divide | 9 nodes: gooch_shade, line, place2d, range, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, triplanarprojection |
| convert | 9 nodes: convert, crosshatch, grid, hsvadjust, randomcolor, randomfloat, tiledcircles, tiledcloverleafs, tiledhexagons |
| ifgreater | 9 nodes: circle, crosshatch, grid, hexagon, line, refract, tiledcircles, tiledcloverleafs, tiledhexagons |
| combine2 | 8 nodes: cloverleaf, crosshatch, hexagon, randomfloat, tiledcircles, tiledcloverleafs, tiledhexagons, triplanarprojection |
| absval | 7 nodes: facingratio, grid, hexagon, range, safepower, trianglewave, triplanarprojection |
| ifequal | 7 nodes: crosshatch, facingratio, grid, range, tiledcircles, tiledcloverleafs, tiledhexagons |
| modulo | 7 nodes: checkerboard, crosshatch, grid, tiledcircles, tiledcloverleafs, tiledhexagons, trianglewave |
| max | 6 nodes: cloverleaf, crosshatch, gooch_shade, tiledcircles, tiledcloverleafs, tiledhexagons |
| mix | 5 nodes: checkerboard, gooch_shade, overlay, ramp4, saturate |
| power | 5 nodes: colorcorrect, gooch_shade, range, safepower, triplanarprojection |
| switch | 5 nodes: extract, place2d, triplanarprojection, unifiednoise2d, unifiednoise3d |
| clamp | 5 nodes: hexagon, line, ramp4, range, triplanarprojection |
| range | 5 nodes: colorcorrect, randomcolor, randomfloat, unifiednoise2d, unifiednoise3d |
| combine3 | 4 nodes: colorcorrect, overlay, randomcolor, unifiednoise2d |
| swizzle | 4 nodes: extract, separate2, separate3, separate4 |
| extract | 3 nodes: convert, ramp4, triplanarprojection |
| separate3 | 3 nodes: colorcorrect, overlay, triplanarprojection |
| fractal3d | 3 nodes: fractal3d, unifiednoise2d, unifiednoise3d |
| normalize | 2 nodes: gooch_shade, triplanarprojection |
| image | 2 nodes: tiledimage, triplanarprojection |
| noise2d | 2 nodes: noise2d, unifiednoise2d |
| noise3d | 2 nodes: noise3d, unifiednoise3d |
| cellnoise2d | 2 nodes: randomfloat, unifiednoise2d |
| rotate2d | 2 nodes: place2d, unifiednoise2d |
| hsvtorgb | 2 nodes: hsvadjust, randomcolor |
| circle | 2 nodes: cloverleaf, tiledcircles |
| min | 2 nodes: grid, hexagon |
| sqrt | 2 nodes: hexagon, refract |
| sign | 2 nodes: range, safepower |
| separate4 | 2 nodes: colorcorrect, overlay |
| combine4 | 2 nodes: colorcorrect, overlay |
| invert | 1 nodes: facingratio |
| normal | 1 nodes: gooch_shade |
| viewdirection | 1 nodes: gooch_shade |
| reflect | 1 nodes: gooch_shade |
| worleynoise2d | 1 nodes: unifiednoise2d |
| separate2 | 1 nodes: unifiednoise2d |
| cellnoise3d | 1 nodes: unifiednoise3d |
| worleynoise3d | 1 nodes: unifiednoise3d |
| rotate3d | 1 nodes: unifiednoise3d |
| ceil | 1 nodes: randomcolor |
| randomfloat | 1 nodes: randomcolor |
| randomcolor | 1 nodes: randomcolor |
| floor | 1 nodes: checkerboard |
| distance | 1 nodes: line |
| constant | 1 nodes: hexagon |
| line | 1 nodes: crosshatch |
| cloverleaf | 1 nodes: tiledcloverleafs |
| hexagon | 1 nodes: tiledhexagons |
| heighttonormal | 1 nodes: bump |
| normalmap | 1 nodes: bump |
| magnitude | 1 nodes: distance |
| smoothstep | 1 nodes: smoothstep |
| remap | 1 nodes: range |
| rgbtohsv | 1 nodes: hsvadjust |
| luminance | 1 nodes: saturate |
| contrast | 1 nodes: colorcorrect |
| saturate | 1 nodes: colorcorrect |
| hsvadjust | 1 nodes: colorcorrect |
| colorcorrect | 1 nodes: colorcorrect |
| ifgreatereq | 1 nodes: overlay |
| overlay | 1 nodes: overlay |
| surface_unlit | 1 nodes: convert |

| Non-Graph Nodes: 99 |
| --- |
|absval, acos, add, ambientocclusion, asin, atan2, bitangent, blur, burn, ceil, cellnoise2d, cellnoise3d, clamp, combine2, combine3, combine4, constant, convert, cos, creatematrix, crossproduct, determinant, difference, disjointover, divide, dodge, dot, dotproduct, exp, floor, fractal3d, frame, geomcolor, geompropvalue, heighttonormal, hsvtorgb, ifequal, ifgreater, ifgreatereq, image, in, inside, invert, invertmatrix, ln, luminance, magnitude, mask, matte, max, min, minus, mix, modulo, multiply, noise2d, noise3d, normal, normalize, normalmap, out, outside, over, plus, position, power, premult, ramplr, ramptb, remap, rgbtohsv, rotate2d, rotate3d, round, screen, sign, sin, smoothstep, splitlr, splittb, sqrt, subtract, surface_unlit, surfacematerial, switch, swizzle, tan, tangent, texcoord, time, transformmatrix, transformnormal, transformpoint, transformvector, transpose, unpremult, viewdirection, worleynoise2d, worleynoise3d|

| Unimplemented Nodes 2 |
| --- |
|curveadjust, volumematerial|

## Node Usage Results for "pbrlib, bxdf"

In [19]:
nodes_use, nodes_used_by, nongraph_nodes, unimplemented = getNodeDefUsage(doc, ['pbrlib', 'bxdf'])

printNodeDefUsage(nodes_use, nodes_used_by, nongraph_nodes, unimplemented)

Skip unknown node type:  dielectric_thinfilm_bsdf
Skip unknown node type:  metal_thinfilm_bsdf
Skip unknown node type:  thin_film_bsdf
Skip unknown node type:  thin_film_bsdf
Skip unknown node type:  thin_film_bsdf
Skip unknown node type:  thin_film_bsdf


| Node Definition | Uses |
| --- | --- |
| open_pbr_surface | 27 nodes: add, clamp, convert, dielectric_bsdf, divide, generalized_schlick_bsdf, generalized_schlick_edf, ifgreater, layer, luminance, max, min, mix, multiply, normalize, oren_nayar_diffuse_bsdf, power, rotate3d, roughness_anisotropy, sheen_bsdf, sign, sqrt, subsurface_bsdf, subtract, surface, translucent_bsdf, uniform_edf |
| standard_surface | 25 nodes: add, artistic_ior, clamp, conductor_bsdf, convert, dielectric_bsdf, divide, generalized_schlick_edf, ifgreater, layer, luminance, max, mix, multiply, normalize, oren_nayar_diffuse_bsdf, power, rotate3d, roughness_anisotropy, sheen_bsdf, subsurface_bsdf, subtract, surface, translucent_bsdf, uniform_edf |
| gltf_pbr | 21 nodes: add, anisotropic_vdf, convert, dielectric_bsdf, divide, extract, generalized_schlick_bsdf, ifequal, ifgreatereq, layer, ln, max, min, mix, multiply, oren_nayar_diffuse_bsdf, roughness_anisotropy, sheen_bsdf, subtract, surface, uniform_edf |
| UsdPreviewSurface | 17 nodes: add, artistic_ior, conductor_bsdf, convert, dielectric_bsdf, divide, generalized_schlick_bsdf, ifgreatereq, layer, mix, multiply, normalmap, oren_nayar_diffuse_bsdf, roughness_anisotropy, subtract, surface, uniform_edf |
| LamaConductor | 17 nodes: add, artistic_ior, clamp, combine2, conductor_bsdf, convert, divide, ifgreater, ifgreatereq, layer, max, multiply, normalize, power, rotate3d, subtract, switch |
| LamaDielectric | 17 nodes: add, anisotropic_vdf, artistic_ior, clamp, combine2, convert, dielectric_bsdf, divide, ifgreatereq, layer, max, multiply, normalize, power, rotate3d, subtract, switch |
| standard_surface_to_gltf_pbr | 7 nodes: divide, dot, dotproduct, ifequal, ifgreater, mix, multiply |
| standard_surface_to_UsdPreviewSurface | 6 nodes: divide, dot, dotproduct, mix, multiply, subtract |
| gltf_colorimage | 5 nodes: combine3, dot, gltf_image, multiply, separate4 |
| gltf_normalmap | 5 nodes: divide, image, multiply, normalmap, place2d |
| gltf_image | 4 nodes: divide, image, multiply, place2d |
| LamaSheen | 4 nodes: add, multiply, power, sheen_bsdf |
| gltf_iridescence_thickness | 3 nodes: extract, gltf_image, mix |
| UsdUVTexture | 3 nodes: add, image, multiply |
| LamaSSS | 3 nodes: convert, multiply, subsurface_bsdf |
| LamaAdd | 2 nodes: add, multiply |
| LamaDiffuse | 2 nodes: multiply, oren_nayar_diffuse_bsdf |
| LamaLayer | 2 nodes: layer, multiply |
| glossiness_anisotropy | 2 nodes: invert, roughness_anisotropy |
| UsdPrimvarReader | 1 nodes: geompropvalue |
| UsdTransform2d | 1 nodes: place2d |
| LamaEmission | 1 nodes: uniform_edf |
| LamaMix | 1 nodes: mix |
| LamaTranslucent | 1 nodes: translucent_bsdf |

| Node Definition | Used by |
| --- | --- |
| multiply | 17 nodes: LamaAdd, LamaConductor, LamaDielectric, LamaDiffuse, LamaLayer, LamaSSS, LamaSheen, UsdPreviewSurface, UsdUVTexture, gltf_colorimage, gltf_image, gltf_normalmap, gltf_pbr, open_pbr_surface, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr |
| divide | 10 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, gltf_image, gltf_normalmap, gltf_pbr, open_pbr_surface, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr |
| add | 9 nodes: LamaAdd, LamaConductor, LamaDielectric, LamaSheen, UsdPreviewSurface, UsdUVTexture, gltf_pbr, open_pbr_surface, standard_surface |
| mix | 8 nodes: LamaMix, UsdPreviewSurface, gltf_iridescence_thickness, gltf_pbr, open_pbr_surface, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr |
| convert | 7 nodes: LamaConductor, LamaDielectric, LamaSSS, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| subtract | 7 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface, standard_surface_to_UsdPreviewSurface |
| layer | 7 nodes: LamaConductor, LamaDielectric, LamaLayer, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| roughness_anisotropy | 5 nodes: UsdPreviewSurface, glossiness_anisotropy, gltf_pbr, open_pbr_surface, standard_surface |
| oren_nayar_diffuse_bsdf | 5 nodes: LamaDiffuse, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| dielectric_bsdf | 5 nodes: LamaDielectric, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| max | 5 nodes: LamaConductor, LamaDielectric, gltf_pbr, open_pbr_surface, standard_surface |
| uniform_edf | 5 nodes: LamaEmission, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| power | 5 nodes: LamaConductor, LamaDielectric, LamaSheen, open_pbr_surface, standard_surface |
| sheen_bsdf | 4 nodes: LamaSheen, gltf_pbr, open_pbr_surface, standard_surface |
| ifgreatereq | 4 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, gltf_pbr |
| surface | 4 nodes: UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| clamp | 4 nodes: LamaConductor, LamaDielectric, open_pbr_surface, standard_surface |
| rotate3d | 4 nodes: LamaConductor, LamaDielectric, open_pbr_surface, standard_surface |
| normalize | 4 nodes: LamaConductor, LamaDielectric, open_pbr_surface, standard_surface |
| ifgreater | 4 nodes: LamaConductor, open_pbr_surface, standard_surface, standard_surface_to_gltf_pbr |
| artistic_ior | 4 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, standard_surface |
| generalized_schlick_bsdf | 3 nodes: UsdPreviewSurface, gltf_pbr, open_pbr_surface |
| dot | 3 nodes: gltf_colorimage, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr |
| image | 3 nodes: UsdUVTexture, gltf_image, gltf_normalmap |
| place2d | 3 nodes: UsdTransform2d, gltf_image, gltf_normalmap |
| translucent_bsdf | 3 nodes: LamaTranslucent, open_pbr_surface, standard_surface |
| subsurface_bsdf | 3 nodes: LamaSSS, open_pbr_surface, standard_surface |
| conductor_bsdf | 3 nodes: LamaConductor, UsdPreviewSurface, standard_surface |
| anisotropic_vdf | 2 nodes: LamaDielectric, gltf_pbr |
| min | 2 nodes: gltf_pbr, open_pbr_surface |
| extract | 2 nodes: gltf_iridescence_thickness, gltf_pbr |
| ifequal | 2 nodes: gltf_pbr, standard_surface_to_gltf_pbr |
| gltf_image | 2 nodes: gltf_colorimage, gltf_iridescence_thickness |
| normalmap | 2 nodes: UsdPreviewSurface, gltf_normalmap |
| generalized_schlick_edf | 2 nodes: open_pbr_surface, standard_surface |
| luminance | 2 nodes: open_pbr_surface, standard_surface |
| switch | 2 nodes: LamaConductor, LamaDielectric |
| combine2 | 2 nodes: LamaConductor, LamaDielectric |
| dotproduct | 2 nodes: standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr |
| ln | 1 nodes: gltf_pbr |
| separate4 | 1 nodes: gltf_colorimage |
| combine3 | 1 nodes: gltf_colorimage |
| sqrt | 1 nodes: open_pbr_surface |
| sign | 1 nodes: open_pbr_surface |
| geompropvalue | 1 nodes: UsdPrimvarReader |
| invert | 1 nodes: glossiness_anisotropy |

| Non-Graph Nodes: 29 |
| --- |
|absorption_vdf, add, anisotropic_vdf, artistic_ior, blackbody, burley_diffuse_bsdf, conductor_bsdf, conical_edf, dielectric_bsdf, disney_brdf_2012, disney_bsdf_2015, displacement, generalized_schlick_bsdf, generalized_schlick_edf, layer, light, measured_edf, mix, multiply, oren_nayar_diffuse_bsdf, roughness_anisotropy, roughness_dual, sheen_bsdf, subsurface_bsdf, surface, thin_surface, translucent_bsdf, uniform_edf, volume|

## Node Usage Results for all Standard Library Nodes

In [20]:
nodes_use, nodes_used_by, nongraph_nodes, unimplemented = getNodeDefUsage(doc)

printNodeDefUsage(nodes_use, nodes_used_by, nongraph_nodes, unimplemented)

Skip unknown node type:  dielectric_thinfilm_bsdf
Skip unknown node type:  metal_thinfilm_bsdf
Skip unknown node type:  thin_film_bsdf
Skip unknown node type:  thin_film_bsdf
Skip unknown node type:  thin_film_bsdf
Skip unknown node type:  thin_film_bsdf


| Node Definition | Uses |
| --- | --- |
| open_pbr_surface | 27 nodes: add, clamp, convert, dielectric_bsdf, divide, generalized_schlick_bsdf, generalized_schlick_edf, ifgreater, layer, luminance, max, min, mix, multiply, normalize, oren_nayar_diffuse_bsdf, power, rotate3d, roughness_anisotropy, sheen_bsdf, sign, sqrt, subsurface_bsdf, subtract, surface, translucent_bsdf, uniform_edf |
| standard_surface | 25 nodes: add, artistic_ior, clamp, conductor_bsdf, convert, dielectric_bsdf, divide, generalized_schlick_edf, ifgreater, layer, luminance, max, mix, multiply, normalize, oren_nayar_diffuse_bsdf, power, rotate3d, roughness_anisotropy, sheen_bsdf, subsurface_bsdf, subtract, surface, translucent_bsdf, uniform_edf |
| gltf_pbr | 21 nodes: add, anisotropic_vdf, convert, dielectric_bsdf, divide, extract, generalized_schlick_bsdf, ifequal, ifgreatereq, layer, ln, max, min, mix, multiply, oren_nayar_diffuse_bsdf, roughness_anisotropy, sheen_bsdf, subtract, surface, uniform_edf |
| UsdPreviewSurface | 17 nodes: add, artistic_ior, conductor_bsdf, convert, dielectric_bsdf, divide, generalized_schlick_bsdf, ifgreatereq, layer, mix, multiply, normalmap, oren_nayar_diffuse_bsdf, roughness_anisotropy, subtract, surface, uniform_edf |
| LamaConductor | 17 nodes: add, artistic_ior, clamp, combine2, conductor_bsdf, convert, divide, ifgreater, ifgreatereq, layer, max, multiply, normalize, power, rotate3d, subtract, switch |
| LamaDielectric | 17 nodes: add, anisotropic_vdf, artistic_ior, clamp, combine2, convert, dielectric_bsdf, divide, ifgreatereq, layer, max, multiply, normalize, power, rotate3d, subtract, switch |
| triplanarprojection | 13 nodes: absval, add, clamp, combine2, divide, dotproduct, extract, image, multiply, normalize, power, separate3, switch |
| unifiednoise2d | 12 nodes: add, cellnoise2d, combine3, fractal3d, multiply, noise2d, range, rotate2d, separate2, subtract, switch, worleynoise2d |
| gooch_shade | 11 nodes: add, divide, dotproduct, max, mix, multiply, normal, normalize, power, reflect, viewdirection |
| tiledcircles | 11 nodes: add, circle, combine2, convert, divide, ifequal, ifgreater, max, modulo, multiply, subtract |
| tiledcloverleafs | 11 nodes: add, cloverleaf, combine2, convert, divide, ifequal, ifgreater, max, modulo, multiply, subtract |
| tiledhexagons | 11 nodes: add, combine2, convert, divide, hexagon, ifequal, ifgreater, max, modulo, multiply, subtract |
| unifiednoise3d | 10 nodes: add, cellnoise3d, fractal3d, multiply, noise3d, range, rotate3d, subtract, switch, worleynoise3d |
| hexagon | 10 nodes: absval, clamp, combine2, constant, dotproduct, ifgreater, min, multiply, sqrt, subtract |
| crosshatch | 10 nodes: add, combine2, convert, ifequal, ifgreater, line, max, modulo, multiply, subtract |
| grid | 9 nodes: absval, add, convert, ifequal, ifgreater, min, modulo, multiply, subtract |
| range | 8 nodes: absval, clamp, divide, ifequal, multiply, power, remap, sign |
| standard_surface_to_gltf_pbr | 7 nodes: divide, dot, dotproduct, ifequal, ifgreater, mix, multiply |
| line | 7 nodes: clamp, distance, divide, dotproduct, ifgreater, multiply, subtract |
| standard_surface_to_UsdPreviewSurface | 6 nodes: divide, dot, dotproduct, mix, multiply, subtract |
| checkerboard | 6 nodes: dotproduct, floor, mix, modulo, multiply, subtract |
| refract | 6 nodes: add, dotproduct, ifgreater, multiply, sqrt, subtract |
| gltf_colorimage | 5 nodes: combine3, dot, gltf_image, multiply, separate4 |
| gltf_normalmap | 5 nodes: divide, image, multiply, normalmap, place2d |
| facingratio | 5 nodes: absval, dotproduct, ifequal, invert, multiply |
| cloverleaf | 5 nodes: add, circle, combine2, max, subtract |
| place2d | 5 nodes: add, divide, rotate2d, subtract, switch |
| hsvadjust | 5 nodes: add, convert, hsvtorgb, multiply, rgbtohsv |
| colorcorrect | 5 nodes: colorcorrect, combine3, combine4, separate3, separate4 |
| gltf_image | 4 nodes: divide, image, multiply, place2d |
| LamaSheen | 4 nodes: add, multiply, power, sheen_bsdf |
| tiledimage | 4 nodes: divide, image, multiply, subtract |
| randomfloat | 4 nodes: cellnoise2d, combine2, convert, range |
| circle | 4 nodes: dotproduct, ifgreater, multiply, subtract |
| safepower | 4 nodes: absval, multiply, power, sign |
| gltf_iridescence_thickness | 3 nodes: extract, gltf_image, mix |
| UsdUVTexture | 3 nodes: add, image, multiply |
| LamaSSS | 3 nodes: convert, multiply, subsurface_bsdf |
| g18_rec709_to_lin_rec709 | 3 nodes: combine4, convert, g18_rec709_to_lin_rec709 |
| g22_rec709_to_lin_rec709 | 3 nodes: combine4, convert, g22_rec709_to_lin_rec709 |
| rec709_display_to_lin_rec709 | 3 nodes: combine4, convert, rec709_display_to_lin_rec709 |
| acescg_to_lin_rec709 | 3 nodes: acescg_to_lin_rec709, combine4, convert |
| g22_ap1_to_lin_rec709 | 3 nodes: combine4, convert, g22_ap1_to_lin_rec709 |
| srgb_texture_to_lin_rec709 | 3 nodes: combine4, convert, srgb_texture_to_lin_rec709 |
| lin_adobergb_to_lin_rec709 | 3 nodes: combine4, convert, lin_adobergb_to_lin_rec709 |
| adobergb_to_lin_rec709 | 3 nodes: adobergb_to_lin_rec709, combine4, convert |
| srgb_displayp3_to_lin_rec709 | 3 nodes: combine4, convert, srgb_displayp3_to_lin_rec709 |
| lin_displayp3_to_lin_rec709 | 3 nodes: combine4, convert, lin_displayp3_to_lin_rec709 |
| ramp4 | 3 nodes: clamp, extract, mix |
| trianglewave | 3 nodes: absval, modulo, subtract |
| reflect | 3 nodes: dotproduct, multiply, subtract |
| contrast | 3 nodes: add, multiply, subtract |
| overlay | 3 nodes: combine4, overlay, separate4 |
| LamaAdd | 2 nodes: add, multiply |
| LamaDiffuse | 2 nodes: multiply, oren_nayar_diffuse_bsdf |
| LamaLayer | 2 nodes: layer, multiply |
| glossiness_anisotropy | 2 nodes: invert, roughness_anisotropy |
| randomcolor | 2 nodes: convert, randomcolor |
| bump | 2 nodes: heighttonormal, normalmap |
| distance | 2 nodes: magnitude, subtract |
| saturate | 2 nodes: luminance, mix |
| convert | 2 nodes: convert, surface_unlit |
| extract | 2 nodes: switch, swizzle |
| UsdPrimvarReader | 1 nodes: geompropvalue |
| UsdTransform2d | 1 nodes: place2d |
| LamaEmission | 1 nodes: uniform_edf |
| LamaMix | 1 nodes: mix |
| LamaTranslucent | 1 nodes: translucent_bsdf |
| noise2d | 1 nodes: noise2d |
| noise3d | 1 nodes: noise3d |
| fractal3d | 1 nodes: fractal3d |
| smoothstep | 1 nodes: smoothstep |
| separate2 | 1 nodes: swizzle |
| separate3 | 1 nodes: swizzle |
| separate4 | 1 nodes: swizzle |

| Node Definition | Used by |
| --- | --- |
| multiply | 41 nodes: LamaAdd, LamaConductor, LamaDielectric, LamaDiffuse, LamaLayer, LamaSSS, LamaSheen, UsdPreviewSurface, UsdUVTexture, checkerboard, circle, colorcorrect, contrast, crosshatch, facingratio, gltf_colorimage, gltf_image, gltf_normalmap, gltf_pbr, gooch_shade, grid, hexagon, hsvadjust, line, open_pbr_surface, overlay, randomfloat, range, reflect, refract, safepower, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, triplanarprojection, unifiednoise2d, unifiednoise3d |
| subtract | 28 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, checkerboard, circle, cloverleaf, colorcorrect, contrast, crosshatch, distance, gltf_pbr, grid, hexagon, line, open_pbr_surface, overlay, place2d, reflect, refract, standard_surface, standard_surface_to_UsdPreviewSurface, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, trianglewave, unifiednoise2d, unifiednoise3d |
| convert | 26 nodes: LamaConductor, LamaDielectric, LamaSSS, UsdPreviewSurface, acescg_to_lin_rec709, adobergb_to_lin_rec709, convert, crosshatch, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, gltf_pbr, grid, hsvadjust, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, open_pbr_surface, randomcolor, randomfloat, rec709_display_to_lin_rec709, srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709, standard_surface, tiledcircles, tiledcloverleafs, tiledhexagons |
| add | 26 nodes: LamaAdd, LamaConductor, LamaDielectric, LamaSheen, UsdPreviewSurface, UsdUVTexture, cloverleaf, colorcorrect, contrast, crosshatch, gltf_pbr, gooch_shade, grid, hsvadjust, open_pbr_surface, place2d, randomcolor, refract, srgb_texture_to_lin_rec709, standard_surface, tiledcircles, tiledcloverleafs, tiledhexagons, triplanarprojection, unifiednoise2d, unifiednoise3d |
| divide | 21 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, adobergb_to_lin_rec709, gltf_image, gltf_normalmap, gltf_pbr, gooch_shade, line, open_pbr_surface, place2d, range, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, triplanarprojection |
| max | 17 nodes: LamaConductor, LamaDielectric, adobergb_to_lin_rec709, cloverleaf, crosshatch, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, gltf_pbr, gooch_shade, open_pbr_surface, rec709_display_to_lin_rec709, srgb_texture_to_lin_rec709, standard_surface, tiledcircles, tiledcloverleafs, tiledhexagons |
| power | 16 nodes: LamaConductor, LamaDielectric, LamaSheen, adobergb_to_lin_rec709, colorcorrect, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, gooch_shade, open_pbr_surface, range, rec709_display_to_lin_rec709, safepower, srgb_texture_to_lin_rec709, standard_surface, triplanarprojection |
| mix | 14 nodes: LamaMix, UsdPreviewSurface, checkerboard, gltf_iridescence_thickness, gltf_pbr, gooch_shade, open_pbr_surface, overlay, ramp4, saturate, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr |
| ifgreater | 14 nodes: LamaConductor, circle, crosshatch, grid, hexagon, line, open_pbr_surface, refract, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_gltf_pbr, tiledcircles, tiledcloverleafs, tiledhexagons |
| combine4 | 12 nodes: acescg_to_lin_rec709, adobergb_to_lin_rec709, colorcorrect, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, overlay, rec709_display_to_lin_rec709, srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709 |
| dotproduct | 11 nodes: checkerboard, circle, facingratio, gooch_shade, hexagon, line, reflect, refract, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, triplanarprojection |
| combine2 | 10 nodes: LamaConductor, LamaDielectric, cloverleaf, crosshatch, hexagon, randomfloat, tiledcircles, tiledcloverleafs, tiledhexagons, triplanarprojection |
| ifequal | 9 nodes: crosshatch, facingratio, gltf_pbr, grid, range, standard_surface_to_gltf_pbr, tiledcircles, tiledcloverleafs, tiledhexagons |
| clamp | 9 nodes: LamaConductor, LamaDielectric, hexagon, line, open_pbr_surface, ramp4, range, standard_surface, triplanarprojection |
| layer | 7 nodes: LamaConductor, LamaDielectric, LamaLayer, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| switch | 7 nodes: LamaConductor, LamaDielectric, extract, place2d, triplanarprojection, unifiednoise2d, unifiednoise3d |
| absval | 7 nodes: facingratio, grid, hexagon, range, safepower, trianglewave, triplanarprojection |
| modulo | 7 nodes: checkerboard, crosshatch, grid, tiledcircles, tiledcloverleafs, tiledhexagons, trianglewave |
| combine3 | 6 nodes: colorcorrect, gltf_colorimage, overlay, randomcolor, srgb_texture_to_lin_rec709, unifiednoise2d |
| normalize | 6 nodes: LamaConductor, LamaDielectric, gooch_shade, open_pbr_surface, standard_surface, triplanarprojection |
| constant | 6 nodes: acescg_to_lin_rec709, hexagon, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709 |
| roughness_anisotropy | 5 nodes: UsdPreviewSurface, glossiness_anisotropy, gltf_pbr, open_pbr_surface, standard_surface |
| oren_nayar_diffuse_bsdf | 5 nodes: LamaDiffuse, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| dielectric_bsdf | 5 nodes: LamaDielectric, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| extract | 5 nodes: convert, gltf_iridescence_thickness, gltf_pbr, ramp4, triplanarprojection |
| uniform_edf | 5 nodes: LamaEmission, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| ifgreatereq | 5 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, gltf_pbr, overlay |
| image | 5 nodes: UsdUVTexture, gltf_image, gltf_normalmap, tiledimage, triplanarprojection |
| rotate3d | 5 nodes: LamaConductor, LamaDielectric, open_pbr_surface, standard_surface, unifiednoise3d |
| range | 5 nodes: colorcorrect, randomcolor, randomfloat, unifiednoise2d, unifiednoise3d |
| min | 4 nodes: gltf_pbr, grid, hexagon, open_pbr_surface |
| sheen_bsdf | 4 nodes: LamaSheen, gltf_pbr, open_pbr_surface, standard_surface |
| surface | 4 nodes: UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| artistic_ior | 4 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, standard_surface |
| transformmatrix | 4 nodes: acescg_to_lin_rec709, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, srgb_displayp3_to_lin_rec709 |
| swizzle | 4 nodes: extract, separate2, separate3, separate4 |
| generalized_schlick_bsdf | 3 nodes: UsdPreviewSurface, gltf_pbr, open_pbr_surface |
| separate4 | 3 nodes: colorcorrect, gltf_colorimage, overlay |
| dot | 3 nodes: gltf_colorimage, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr |
| place2d | 3 nodes: UsdTransform2d, gltf_image, gltf_normalmap |
| normalmap | 3 nodes: UsdPreviewSurface, bump, gltf_normalmap |
| translucent_bsdf | 3 nodes: LamaTranslucent, open_pbr_surface, standard_surface |
| subsurface_bsdf | 3 nodes: LamaSSS, open_pbr_surface, standard_surface |
| sqrt | 3 nodes: hexagon, open_pbr_surface, refract |
| sign | 3 nodes: open_pbr_surface, range, safepower |
| luminance | 3 nodes: open_pbr_surface, saturate, standard_surface |
| conductor_bsdf | 3 nodes: LamaConductor, UsdPreviewSurface, standard_surface |
| separate3 | 3 nodes: colorcorrect, overlay, triplanarprojection |
| fractal3d | 3 nodes: fractal3d, unifiednoise2d, unifiednoise3d |
| anisotropic_vdf | 2 nodes: LamaDielectric, gltf_pbr |
| gltf_image | 2 nodes: gltf_colorimage, gltf_iridescence_thickness |
| generalized_schlick_edf | 2 nodes: open_pbr_surface, standard_surface |
| acescg_to_lin_rec709 | 2 nodes: acescg_to_lin_rec709, g22_ap1_to_lin_rec709 |
| srgb_texture_to_lin_rec709 | 2 nodes: srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709 |
| lin_adobergb_to_lin_rec709 | 2 nodes: adobergb_to_lin_rec709, lin_adobergb_to_lin_rec709 |
| invert | 2 nodes: facingratio, glossiness_anisotropy |
| noise2d | 2 nodes: noise2d, unifiednoise2d |
| noise3d | 2 nodes: noise3d, unifiednoise3d |
| cellnoise2d | 2 nodes: randomfloat, unifiednoise2d |
| rotate2d | 2 nodes: place2d, unifiednoise2d |
| hsvtorgb | 2 nodes: hsvadjust, randomcolor |
| circle | 2 nodes: cloverleaf, tiledcircles |
| ln | 1 nodes: gltf_pbr |
| geompropvalue | 1 nodes: UsdPrimvarReader |
| g18_rec709_to_lin_rec709 | 1 nodes: g18_rec709_to_lin_rec709 |
| g22_rec709_to_lin_rec709 | 1 nodes: g22_rec709_to_lin_rec709 |
| rec709_display_to_lin_rec709 | 1 nodes: rec709_display_to_lin_rec709 |
| g22_ap1_to_lin_rec709 | 1 nodes: g22_ap1_to_lin_rec709 |
| adobergb_to_lin_rec709 | 1 nodes: adobergb_to_lin_rec709 |
| srgb_displayp3_to_lin_rec709 | 1 nodes: srgb_displayp3_to_lin_rec709 |
| lin_displayp3_to_lin_rec709 | 1 nodes: lin_displayp3_to_lin_rec709 |
| normal | 1 nodes: gooch_shade |
| viewdirection | 1 nodes: gooch_shade |
| reflect | 1 nodes: gooch_shade |
| worleynoise2d | 1 nodes: unifiednoise2d |
| separate2 | 1 nodes: unifiednoise2d |
| cellnoise3d | 1 nodes: unifiednoise3d |
| worleynoise3d | 1 nodes: unifiednoise3d |
| ceil | 1 nodes: randomcolor |
| randomfloat | 1 nodes: randomcolor |
| randomcolor | 1 nodes: randomcolor |
| floor | 1 nodes: checkerboard |
| distance | 1 nodes: line |
| line | 1 nodes: crosshatch |
| cloverleaf | 1 nodes: tiledcloverleafs |
| hexagon | 1 nodes: tiledhexagons |
| heighttonormal | 1 nodes: bump |
| magnitude | 1 nodes: distance |
| smoothstep | 1 nodes: smoothstep |
| remap | 1 nodes: range |
| rgbtohsv | 1 nodes: hsvadjust |
| contrast | 1 nodes: colorcorrect |
| saturate | 1 nodes: colorcorrect |
| hsvadjust | 1 nodes: colorcorrect |
| colorcorrect | 1 nodes: colorcorrect |
| overlay | 1 nodes: overlay |
| surface_unlit | 1 nodes: convert |

| Non-Graph Nodes: 128 |
| --- |
|absorption_vdf, absval, acos, add, ambientocclusion, anisotropic_vdf, artistic_ior, asin, atan2, bitangent, blackbody, blur, burley_diffuse_bsdf, burn, ceil, cellnoise2d, cellnoise3d, clamp, combine2, combine3, combine4, conductor_bsdf, conical_edf, constant, convert, cos, creatematrix, crossproduct, determinant, dielectric_bsdf, difference, directional_light, disjointover, disney_brdf_2012, disney_bsdf_2015, displacement, divide, dodge, dot, dotproduct, exp, floor, fractal3d, frame, generalized_schlick_bsdf, generalized_schlick_edf, geomcolor, geompropvalue, heighttonormal, hsvtorgb, ifequal, ifgreater, ifgreatereq, image, in, inside, invert, invertmatrix, layer, light, ln, luminance, magnitude, mask, matte, max, measured_edf, min, minus, mix, modulo, multiply, noise2d, noise3d, normal, normalize, normalmap, oren_nayar_diffuse_bsdf, out, outside, over, plus, point_light, position, power, premult, ramplr, ramptb, remap, rgbtohsv, rotate2d, rotate3d, roughness_anisotropy, roughness_dual, round, screen, sheen_bsdf, sign, sin, smoothstep, splitlr, splittb, spot_light, sqrt, subsurface_bsdf, subtract, surface, surface_unlit, surfacematerial, switch, swizzle, tan, tangent, texcoord, thin_surface, time, transformmatrix, transformnormal, transformpoint, transformvector, translucent_bsdf, transpose, uniform_edf, unpremult, viewdirection, volume, worleynoise2d, worleynoise3d|

| Unimplemented Nodes 2 |
| --- |
|curveadjust, volumematerial|